In [1]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import seaborn as sns


import torchvision.datasets as tvds

In [33]:
train_set = tvds.EMNIST("./datasets", split='digits', train=True, download=True)
test_set = tvds.EMNIST("./datasets", split='digits', download=True)

In [34]:
train_set.data.shape

torch.Size([240000, 28, 28])

In [35]:
class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()

        self.act = nn.ReLU()
        self.classifier_act = nn.Softmax(dim=1)
        # ((in+2p-d*(k-1)-1) / s) + 1
        self.conv1 = nn.Conv2d(1, 16, 3) # 26x26 
        self.conv2 = nn.Conv2d(16, 32, 5, stride = 2) # 11x11
        self.conv3 = nn.Conv2d(32, 16, 3, stride = 2) # 5x5

        self.classifier = nn.Linear(400, 62)
    
    def forward(self, x):
        x = self.act(self.conv1(x))
        x = self.act(self.conv2(x))
        x = self.act(self.conv3(x))
        x = x.view(x.size(0), -1) # flatten the data

        outs = self.classifier(x)
        return self.classifier_act(outs)

    def set_config(self, optimizer, loss_fn, device="cpu"):
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.device = device
        self.to(device)


    def train_step(self, x, y):
        outs = self.forward(x)
        loss = self.loss_fn(outs, y)
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.cpu().item()

model = Baseline()

model(torch.rand((1, 1, 28, 28)))

tensor([[0.0177, 0.0147, 0.0170, 0.0163, 0.0148, 0.0165, 0.0159, 0.0164, 0.0158,
         0.0153, 0.0153, 0.0163, 0.0161, 0.0163, 0.0170, 0.0171, 0.0174, 0.0158,
         0.0165, 0.0159, 0.0177, 0.0160, 0.0154, 0.0156, 0.0152, 0.0165, 0.0150,
         0.0156, 0.0170, 0.0160, 0.0152, 0.0155, 0.0152, 0.0179, 0.0153, 0.0158,
         0.0173, 0.0157, 0.0172, 0.0156, 0.0160, 0.0157, 0.0171, 0.0149, 0.0168,
         0.0163, 0.0157, 0.0155, 0.0158, 0.0177, 0.0163, 0.0164, 0.0165, 0.0145,
         0.0165, 0.0156, 0.0172, 0.0153, 0.0151, 0.0166, 0.0167, 0.0169]],
       grad_fn=<SoftmaxBackward0>)

In [36]:
x = torch.utils.data.DataLoader(train_set.data, batch_size=256)
for i in x:
    print(i.shape)
    break

torch.Size([256, 28, 28])


In [37]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,dataset):
        super(Dataset, self).__init__()
        self.x = dataset.data
        self.y = dataset.targets
        self.classes = dataset.classes
        self.class_to_idx = dataset.class_to_idx

        assert len(self.x)==len(self.y), "Number of samples doesn't match!"

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = list(idx)
        
        x = self.x[idx]
        y = self.y[idx]

        x = torch.unsqueeze(x, 0) if type(idx)==int else torch.unsqueeze(x, 1)

        return (x, y)


    def __len__(self):
        return len(self.x)

train_dataset = Dataset(train_set)

In [38]:
from tqdm import tqdm
import time

def split_data(dataset, ratio=.3):
    N = int(len(dataset)*ratio)
    split1, split2 = torch.utils.data.random_split(dataset, [len(dataset) - N, N])
    return split1, split2

def validate(loader, model):
    losses = []
    with torch.no_grad():
        for batch in loader:
            x_batch, y_batch = batch[0].to(model.device, dtype=torch.float32), batch[1].to(model.device)
            outs = model(x_batch)
            loss = model.loss_fn(outs, y_batch)
            losses.append(loss.cpu().item())
    return sum(losses)/len(losses)

def train_model(train_loader, val_loader, model, loss_fn, optimizer, device='cpu', n_epochs=1, verbose=0):
    model.set_config(optimizer=optimizer, loss_fn=loss_fn, device=device)
    train_losses = []
    val_losses = []


    for i in tqdm(range(n_epochs)):
        epoch_train_loss = 0

        t0 = time.time()

        for batch in tqdm(train_loader):
            x_batch, y_batch = batch[0].to(model.device, dtype=torch.float32), batch[1].to(model.device)
            train_loss = model.train_step(x_batch, y_batch)
            epoch_train_loss += train_loss / len(train_loader)
        
        # validate
        epoch_val_loss = validate(val_loader, model)

        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        t1 = time.time()
        print("Training loss: %.4f | Validation loss: %.4f | took %.4f secs"%(epoch_train_loss, epoch_val_loss, t1-t0))

    return train_losses, val_losses


BATCH_SIZE = 512
train_data, val_data = split_data(train_dataset)
train_loader, val_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE), torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE)

model = Baseline()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

device = torch.device("cuda:0" if torch.cuda.is_available else 'cpu')
print("Training on: %s"%device)
losses = train_model(train_loader, val_loader, model, loss_fn, optimizer, device=device)


Training on: cuda:0


100%|██████████| 1/1 [00:11<00:00, 11.65s/it]

Training loss: 3.6940 | Validation loss: 3.6633 | took 11.6496 secs


In [11]:
for i in train_loader:
    print(len(i[1]))
    break

512
